In [14]:
from tvpblp import TvPBLP, sample_data_path
%load_ext autoreload
%autoreload 2


import pandas as pd
import numpy as np
from numpy.linalg import inv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
df

,Unnamed: 0,product,price,share,weight,calories,sugar,protein,distance,diesel,...,month_31,month_32,month_33,month_34,month_35,month_36,product_2,product_3,product_4,product_5
0,0,1,0.784537,0.159556,170,0.005190,0.000450,0.000208,1.394058,3.128164,...,False,False,False,False,False,False,False,False,False,False
1,1,2,1.088631,0.159618,150,0.003556,0.000000,0.000622,1.343870,3.128164,...,False,False,False,False,False,False,True,False,False,False
2,2,3,1.034824,0.158163,150,0.004444,0.000000,0.000667,1.020209,3.128164,...,False,False,False,False,False,False,False,True,False,False
3,3,4,0.835741,0.137870,170,0.005190,0.000381,0.000173,1.281764,3.128164,...,False,False,False,False,False,False,False,False,True,False
4,4,5,0.480414,0.134294,113,0.007048,0.000627,0.000313,1.116590,3.128164,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27579,27579,1,0.791183,0.188956,170,0.005190,0.000450,0.000208,1.095799,3.954709,...,False,False,False,False,False,True,False,False,False,False
27580,27580,2,1.084542,0.169545,150,0.003556,0.000000,0.000622,1.272327,3.954709,...,False,False,False,False,False,True,True,False,False,False
27581,27581,3,1.061886,0.146912,150,0.004444,0.000000,0.000667,1.439315,3.954709,...,False,False,False,False,False,True,False,True,False,False
27582,27582,4,0.909000,0.137241,170,0.005190,0.000381,0.000173,1.533821,3.954709,...,False,False,False,False,False,True,False,False,True,False


In [19]:
model = TvPBLP(df, share_col='share', price_col='price',
lin = lin_columns, random = random_columns,
instruments = ['instrument_1', 'instrument_2'], market_id='market',
product_id='product', integration_points=50, integration_int = [0.1, 0.9], og_share_col='og_share')

NameError: name 'lin_columns' is not defined

In [4]:
W = inv(model.Z.T@model.Z)
model.fit(W, L_init = [[700]])


Starting optimization with L_init diagonal: [700.]
Stopping tolerance: max|ΔL| < 1e-12
  Iter   5: obj = 1.350198e-03, max|ΔL| = 4.731e+02
  Iter  10: obj = 2.036775e-19, max|ΔL| = 3.116e-04
  Converged! max|ΔL| = 3.695e-13 < 1e-12

Optimization finished after 13 iterations
Convergence by L change: True
Success: True
Final objective: 3.061705e-23
Optimal L:
[[152.8598705]]


{'L_opt': array([[152.8598705]]),
 'obj_val': 3.0617051503614194e-23,
 'result':   message: CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
   success: True
    status: 0
       fun: 3.061705150361419e-43
         x: [ 1.529e+02]
       nit: 9
       jac: [ 0.000e+00]
      nfev: 13
      njev: 13
  hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>,
 'n_iter': 13}

In [5]:
list([1,2,3])

[1, 2, 3]